 # Video colorizer, it is colorize in real time.

In [4]:
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dense, Activation, BatchNormalization, Conv2D, BatchNormalization, MaxPooling2D, UpSampling2D, Flatten
import numpy as np
import scipy as sp
from skimage import color
import warnings
import keras
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
# Import own .py files
import data_collector
import image_loader
from nnetwork import weighted_categorical_crossentropy
import nnetwork
from utility_methods import save_plots
from importlib import reload  # Python 3.4+ only.
nnetwork = reload(nnetwork)
import gc


First you need trained weights. After training you can load up in here, and start the colorization.

In [6]:
test_weights = 'O:/ProgrammingSoftwares/anaconda_projects/dp_nagyhazi/tests/20181204_b_model/weights.hdf5'
root_pictures = 'O:/ProgrammingSoftwares/anaconda_projects/dp_nagyhazi/tests/20181204_b_model/test_pics/'
image_folder = 'O:/ProgrammingSoftwares/anaconda_projects/dp_nagyhazi/samples/images/'
distrib = np.load('O:/ProgrammingSoftwares/anaconda_projects/dp_nagyhazi/data/prior_probs.npy')
los = weighted_categorical_crossentropy(distrib,lmb=0.5)
model = keras.models.load_model(test_weights, custom_objects={'loss': los})

In [7]:
import streaming_data
streaming_data = reload(streaming_data)

watcher = streaming_data.StreamingDataGenerator(image_folder,batch_size=1)

O:\ProgrammingSoftwares\anaconda3\envs\tensor-cpu\lib\site-packages\keras_preprocessing\image.py:836: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Length of image list:  28


In [39]:
from imutils.video import FileVideoStream
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import time
import cv2
from matplotlib.pyplot import imshow
from skimage import color
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from skimage import transform, io, color

#out = cv2.VideoWriter('output.avi', -1, 20.0, (224,224))
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('outpy.avi',fourcc, 10, (224,224))

# The video file should be next to this notebook.
filename = 'landscape2.mp4'

print("[INFO] starting video file thread...")
fvs = FileVideoStream(filename).start()
time.sleep(1.0)
 
# start the FPS timer
fps = FPS().start()

[INFO] starting video file thread...


In [40]:
# loop over frames from the video file stream
while fvs.more():
    # grab the frame from the threaded video file stream, resize
    # it, and convert it to grayscale (while still retaining 3
    # channels)
    frame = fvs.read()
    frame = imutils.resize(frame, width=450)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = np.dstack([frame, frame, frame])
    frame = transform.resize(frame , (224,224), preserve_range=False)
    #print(frame)
    
    frame2 = pred_image(model, frame, watcher)
    #print(np.max(frame2))
    # display the size of the queue on the frame
    #cv2.putText(frame2[0], "Queue Size: {}".format(fvs.Q.qsize()),(10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
    # show the frame and update the FPS counter
    
    cv2.imshow("Frame", frame2)
    #out.write(frame2)
    cv2.waitKey(1)
    
    fps.update()

# stop the timer and display FPS information
fps.stop()
print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
cv2.destroyAllWindows()
fvs.stop()

O:\ProgrammingSoftwares\anaconda3\envs\tensor-cpu\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


[INFO] elasped time: 85.11
[INFO] approx. FPS: 1.21


In [14]:
def pred_image(model, X, watcher, Y = None):
    X = X.reshape((1,224,224,3))*255
    inp = X - np.mean(X)
    #plt.imshow(X[0]/255)
    #plt.show()
    y_real = model.predict(inp)
    if (Y is not None):
        y_real = Y

    # Choose the a-b colors from the color classes.
    y_real = np.apply_along_axis(lambda x: watcher.pts_in_hull[np.argmax(x)], axis=3, arr = y_real)[0]
    lightness_3 = transform.resize( X[0] , (56,56), preserve_range=True)
    #print(np.min(y_real[:,:,1]))
    #print(np.max(y_real[:,:,1]))

    lab_im = np.concatenate([lightness_3[:,:,0,np.newaxis]/255*100, y_real[:,:,0,np.newaxis],y_real[:,:,1,np.newaxis]], axis = 2)
    
    rgb_img = color.lab2rgb(lab_im)
    rgb_img = transform.resize( rgb_img , (224,224), preserve_range=False)
    lab_im = color.rgb2lab(rgb_img)
    lab_im = np.concatenate([ X[0,:,:,0,np.newaxis]/255*100, lab_im[:,:,1,np.newaxis],lab_im[:,:,2,np.newaxis]], axis = 2)

    rgb_img = color.lab2rgb(lab_im)
    
    return rgb_img